<img width="8%" alt="Content" src="https://naasai-public.s3.eu-west-3.amazonaws.com/abi-demo/content_creation.png" style="border-radius: 15%">

# Content - Create Assistant plugin

**Tags:** #content #chat #plugin #ai #naaschatplugin #naas #naas_driver

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Description:** This notebook creates the Content Assistant plugin.

## Input

### Import libraries

In [ ]:
import naas
from naas_drivers import naas_chat_plugin, gsheet
import os
import pandas as pd
import json
import requests
import naas_data_product

### Setup variables

In [ ]:
# Inputs
entity_index = "0"
spreadsheet_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", entity_index), "abi_spreadsheet") or ""
sheet_name = "DATASET_POSTS"
name = "Content Assistant"
model = "gpt-4-1106-preview"
temperature = 0.5
description = "Streamline the generation and distribution of content that aligns with the user's or business's brand voice and audience engagement goals."
avatar = "https://naasai-public.s3.eu-west-3.amazonaws.com/abi-demo/content_creation.png"
api_key = naas.secret.get('NAAS_API_TOKEN')
workspace_ids = ""
personal_workspace = True

# Outputs
output_dir = os.path.join("..", "outputs")

## Model

### Helper Functions

#### Create Naas Chat plugin

In [ ]:
def create_plugin(
    name,
    prompt="",
    model="gpt-3.5-turbo-16k",
    temperature=0,
    output_dir=None,
    output_path=None,
    description="",
    avatar="",
    prompt_type="system",
    entity_name=None
):
    # Init
    plugin = {}
    
    # Create output path
    os.makedirs(output_dir, exist_ok=True)
    if not output_path:
        output_path = os.path.join(output_dir, f"{name.lower().replace(' ', '_')}.json")
        
    # Create empty plugin
    if not os.path.exists(output_path):
        with open(output_path, "w") as f:
            json.dump(plugin, f)
        naas.asset.add(output_path, params={"inline": True})
        
    # Get asset URL
    plugin_url = naas.asset.find(output_path)
    
    # Check tokens
    prompt_tokens, max_tokens = naas_chat_plugin.check_tokens(prompt, model)

    # Create JSON
    entity_id = entity_name.lower().replace(' ', '-')
    plugin = {
        "id": f"{name.lower().replace(' ', '-')}-{entity_id}-test",
        "slug": f"{name.lower().replace(' ', '-')}/{entity_id}-test",
        "url": plugin_url,
        "name": f"{name} - {entity_name}",
        "model": model,
        "temperature": temperature,
        "max_tokens": max_tokens,
        "prompt": prompt,
        "commands": [],
        "description": description,
        "avatar": avatar,
        "prompt_type": prompt_type,
    }

    # Save dict to JSON file
    with open(output_path, "w") as f:
        json.dump(plugin, f)
    print(f"💾 Plugin successfully saved. You can use it in your Naas Chat with: {output_path}")
    naas.asset.add(output_path, params={"inline": True})
    return output_path, plugin_url, plugin

#### List Workspaces

In [ ]:
def list_workspaces(
    api_key,
):
    url = "https://api.naas.ai/workspace/"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    response = requests.get(url, headers=headers)
    return response.json()

#### List Workspace plugins

In [ ]:
def list_workspace_plugins(
    api_key,
    workspace_id,
):
    url = f"https://api.naas.ai/workspace/{workspace_id}/plugin"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    response = requests.get(url, headers=headers)
    return response.json()

#### Create Workspace Plugin

In [ ]:
def create_workspace_plugin(
    api_key,
    workspace_id,
    plugin,
):
    url = f"https://api.naas.ai/workspace/{workspace_id}/plugin"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        "workspace_id": workspace_id,
        "payload": json.dumps(plugin),
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()

#### Update Workspace Plugin

In [ ]:
def update_workspace_plugin(
    api_key,
    workspace_id,
    plugin_id,
    plugin,
):
    url = f"https://api.naas.ai/workspace/{workspace_id}/plugin/{plugin_id}"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        "workspace_id": workspace_id,
        "plugin_id": plugin_id,
        "workspace_plugin": {
            "payload": json.dumps(plugin),
        }
    }
    response = requests.put(url, headers=headers, json=data)
    return response.json()

#### Push plugin to Personal workspace

In [ ]:
def push_plugin_to_workspace(
    api_key,
    plugin,
    workspace_ids=[],
    personal_workspace=True
):
    # Init
    if isinstance(workspace_ids, str) and workspace_ids == '':
        workspace_ids = []
        
    # Get workspaces
    workspaces = list_workspaces(api_key)
    
    # Get existing workspace ids
    current_workspace_ids = [workspace.get("id") for workspace in workspaces.get("workspaces")]

    if personal_workspace:
        personal_workspace_id = None
        # Get personal workspace
        for workspace in workspaces.get("workspaces"):
            if workspace.get("is_personal"):
                personal_workspace_id = workspace.get("id")
                break
        if personal_workspace_id not in workspace_ids:
            workspace_ids.append(personal_workspace_id)
    
    for index, workspace_id in enumerate(workspace_ids):
        if workspace_id in current_workspace_ids:
            workspace_id = workspace_id.strip()
            print(f"{index} - Workspace ID:", workspace_id)
            # List existing plugins
            plugins = list_workspace_plugins(
                api_key,
                workspace_id,
            )
            plugin_exist = False
            print("Plugins:", len(plugins.get('workspace_plugins')))
            for i, p in enumerate(plugins.get('workspace_plugins')):
                plugin_id = p.get("id")
                p_json = json.loads(p.get("payload"))
                p_name = p_json.get("name")
                p_slug = p_json.get("slug")
                p_url = p_json.get("url")
                if plugin_url == p_url:
                    plugin_exist = True
                    print("Plugin ID:", plugin_id)
                    print("- Slug:", p_slug)
                    print("- Name:", p_name)
                    print("- URL:", p_url)
                    break

            # Create or update plugin
            if not plugin_exist:
                result = create_workspace_plugin(
                    api_key,
                    workspace_id,
                    plugin,
                )
                print("✅ Plugin successfully created")
            else:
                result = update_workspace_plugin(
                    api_key,
                    workspace_id,
                    plugin_id,
                    plugin,
                )
                print("✅ Plugin successfully updated")
            print()
        else:
            print("❌ Workspace ID does not exist for user.")
    return result

### Get dataset POST

In [ ]:
df = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
print("Rows:", len(df))
df.head(1)

### Create prompt

In [ ]:
# Get variables from data
entity_name = df.loc[0, "ENTITY"]

system_prompt = f"""Act as a Content Assistant who has access to valuable data and insights about the content strategy.
Your role is to manage and optimize the content, ensuring it reaches the target audience effectively. 
On {", ".join(df.SCENARIO.unique())} the '{entity_name}' published {len(df)} contents. 
Here is the list with the post published: {df.to_string()}.
The first message should be about presenting yourself and analyze '{entity_name}' contents published with a maximum of 3 bullet points.
Be casual, but professional. Wait for the first answer from the user, and then start with the first high-level analysis. 
"""

### Create Naas Chat plugin

In [ ]:
print("--> Creating Naas Chat plugin")
plugin_file_path, plugin_url, plugin = create_plugin(
    name=name,
    prompt=system_prompt,
    model=model,
    temperature=temperature,
    output_dir=output_dir,
    description=description,
    avatar=avatar,
    entity_name=entity_name,
)
print()

## Output

### Push to Workspace

In [ ]:
print("--> Push plugin to Personal workspace")
push_plugin_to_workspace(api_key, plugin, workspace_ids, personal_workspace)
print()